In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
from torch.utils.data import TensorDataset
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights   
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score

batch_size = 256
num_workers = 8

In [3]:
# Load the saved fold data from the .pkl file
path = '/home/jaiswalsuraj/suraj_work/Brick-Kilns-Project/data/albk_v2_data/fold_data_delhi.pt'
fold_data = torch.load(path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# save_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/"
save_path = '/home/jaiswalsuraj/suraj_work/Brick-Kilns-Project/data/models/'
# Lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

accuracy_list_train_bang_predict_delhi = []
precision_list_train_bang_predict_delhi = []
recall_list_train_bang_predict_delhi = []
f1_list_train_bang_predict_delhi = []

for fold_info in fold_data:
    fold = fold_info['fold']
    print("Fold: ", fold)
    X_train = fold_info['X_train']
    y_train = fold_info['y_train']
    X_test = fold_info['X_test']
    y_test = fold_info['y_test']
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoader for training and testing
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # Create and train the model
    #print datatype of trainloader
    print("trainloader datatype: ", train_loader.dataset.tensors[1].dtype)
    print("testloader datatype: ", test_loader.dataset.tensors[0].dtype)

    train_model = EfficientNetClassifier    (
        models.efficientnet_b0, EfficientNet_B0_Weights, n_classes=2, activation=nn.ReLU(), dropout=0.1
    ).to(device)
    
    train_model.load_state_dict(torch.load('/home/jaiswalsuraj/suraj_work/Brick-Kilns-Project/data/models/efficientnet_train_bang_fold_1.pth'))
    # Evaluate the model on the test set
    with torch.no_grad():
        pred_classes = train_model.predict_class(
            dataloader=test_loader, batch_size=batch_size, verbose=True
        ).to(device)

    test_labels = y_test.to(device)
    # Calculate and print metrics for each fold
    
    accuracy = accuracy_score( pred_classes,test_labels)
    precision = precision_score(pred_classes,test_labels)
    recall = recall_score(pred_classes,test_labels)
    f1 = f1_score(pred_classes,test_labels)

    print(f"Train model in bang and predict in delhi for fold {fold}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\n")
    # Calculate and store metrics for each fold
    accuracy_list_train_bang_predict_delhi.append(accuracy_score(pred_classes, test_labels))
    precision_list_train_bang_predict_delhi.append(precision_score( pred_classes,test_labels))
    recall_list_train_bang_predict_delhi.append(recall_score(pred_classes,test_labels))
    f1_list_train_bang_predict_delhi.append(f1_score(pred_classes,test_labels))
    
    # Now lets fine tune the model on delhi data
    iter_losses, epoch_losses = train_fn(
        train_model,
        nn.CrossEntropyLoss(),
        dataloader=train_loader,
        lr=3e-4,
        epochs=100,
        verbose=True,
        wandb_log=False,
    )
    torch.save(train_model.state_dict(), save_path + f'efficientnet_train_bang_delhi_fold_{fold}.pth')
    
    # Evaluate the model on the test set
    with torch.no_grad():
        pred_classes = train_model.predict_class(
            dataloader=test_loader, batch_size=batch_size, verbose=True
        ).to(device)

    test_labels = y_test.to(device)
    # Calculate and print metrics for each fold
    
    accuracy = accuracy_score( pred_classes,test_labels)
    precision = precision_score(pred_classes,test_labels)
    recall = recall_score(pred_classes,test_labels)
    f1 = f1_score(pred_classes,test_labels)
    print(f"Train model in bang and fine tune on delhi and predict on delhi for fold {fold}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\n")
    # Calculate and store metrics for each fold
    accuracy_list.append(accuracy_score(pred_classes, test_labels))
    precision_list.append(precision_score( pred_classes,test_labels))
    recall_list.append(recall_score(pred_classes,test_labels))
    f1_list.append(f1_score(pred_classes,test_labels))


Fold:  1
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


/home/jaiswalsuraj/miniconda3/envs/torch_gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 10/10 [00:11<00:00,  1.17s/it]


Train model in bang and predict in delhi for fold 1
Accuracy: 0.9398
Precision: 0.8819
Recall: 0.4866
F1 Score: 0.6272




100%|██████████| 10/10 [00:01<00:00,  7.76it/s]


Train model in bang and fine tune on delhi and predict on delhi for fold 1
Accuracy: 0.9785
Precision: 0.9124
Recall: 0.8774
F1 Score: 0.8945


Fold:  2
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


...

Train model in bang and fine tune on delhi and predict on delhi for fold 3
Accuracy: 0.9769
Precision: 0.8855
Recall: 0.8923
F1 Score: 0.8889


Fold:  4
trainloader datatype:  torch.uint8
testloader datatype:  torch.float32


100%|██████████| 10/10 [00:00<00:00, 11.57it/s]


Train model in bang and predict in delhi for fold 4
Accuracy: 0.9425
Precision: 0.9149
Recall: 0.4943
F1 Score: 0.6418




100%|██████████| 10/10 [00:01<00:00,  7.77it/s]

Train model in bang and fine tune on delhi and predict on delhi for fold 4
Accuracy: 0.9785
Precision: 0.9190
Recall: 0.8697
F1 Score: 0.8937




In [4]:
# Calculate and print the mean of metrics across all folds
print("Mean Accuracy: ", sum(accuracy_list_train_bang_predict_delhi) / len(accuracy_list_train_bang_predict_delhi))
print("Mean Precision: ", sum(precision_list_train_bang_predict_delhi) / len(precision_list_train_bang_predict_delhi))
print("Mean Recall: ", sum(recall_list_train_bang_predict_delhi) / len(recall_list_train_bang_predict_delhi))
print("Mean F1: ", sum(f1_list_train_bang_predict_delhi) / len(f1_list_train_bang_predict_delhi))

Mean Accuracy:  tensor(0.9414, device='cuda:0')
Mean Precision:  tensor(0.8973, device='cuda:0')
Mean Recall:  tensor(0.4933, device='cuda:0')
Mean F1:  tensor(0.6365, device='cuda:0')


In [5]:
# Calculate and print the mean of metrics across all folds
print("Mean Accuracy: ", sum(accuracy_list) / len(accuracy_list))
print("Mean Precision: ", sum(precision_list) / len(precision_list))
print("Mean Recall: ", sum(recall_list) / len(recall_list))
print("Mean F1: ", sum(f1_list) / len(f1_list))

Mean Accuracy:  tensor(0.9780, device='cuda:0')
Mean Precision:  tensor(0.9088, device='cuda:0')
Mean Recall:  tensor(0.8762, device='cuda:0')
Mean F1:  tensor(0.8921, device='cuda:0')
